In [ ]:
%%capture
pip install plotly pandas statsmodels kaleido scipy nbformat jinja2

In [ ]:
# read CSV data

import glob
import re
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import numpy as np
import os.path
import pickle
import scipy
from statistics import mean, stdev
from math import sqrt, log10
from packaging.version import Version

pio.renderers.default = "notebook"
output_directory = '../../output-archive/output-linux-2024-11-08'
figures_directory = '../../../paper-icse-2026-time-travel/images'
default_height = 270

pio.templates['colorblind'] = go.layout.Template(layout_colorway=['#648FFF', '#FE6100', '#785EF0', '#DC267F', '#FFB000'])
pio.templates.default = 'plotly_white+colorblind'

def read_dataframe(stage, dtype={}, usecols=None, file=None):
    if not file:
        file = 'output'
    df = pd.read_csv(f'{output_directory}/{stage}/{file}.csv', dtype=dtype, usecols=usecols)
    if 'committer_date_unix' in df:
        df['committer_date'] = df['committer_date_unix'].apply(lambda d: pd.to_datetime(d, unit='s'))
    return df

def replace_values(df):
    df.replace('kconfigreader', 'KConfigReader', inplace=True)
    df.replace('kmax', 'KClause', inplace=True)

def big_log10(str):
    return log10(int(str)) if not pd.isna(str) and str != '' else pd.NA

def process_model_count(df_solve):
    df_solve['model-count'] = df_solve['model-count'].replace('1', '')
    df_solve['model-count-log10'] = df_solve['model-count'].fillna('').apply(big_log10).replace(0, np.nan)
    df_solve['year'] = df_solve['committer_date'].apply(lambda d: int(d.year))

def peek_dataframe(df, column, message, type='str', filter=['revision', 'architecture', 'extractor']):
    success = df[~df[column].str.contains('NA') if type == 'str' else ~df[column].isna()][filter]
    failure = df[df[column].str.contains('NA') if type == 'str' else df[column].isna()][filter]
    print(f'{message}: {len(success)} successes, {len(failure)} failures')

df_architectures = read_dataframe(f'read-linux-architectures')
df_architectures = df_architectures.sort_values(by='committer_date')
df_architectures['year'] = df_architectures['committer_date'].apply(lambda d: int(d.year))

df_configs = read_dataframe(f'read-linux-configs')
df_configs = df_configs[~df_configs['kconfig-file'].str.contains('/um/')]

df_config_types = read_dataframe(f'read-linux-configs', file='output.types')
df_config_types = df_config_types[~df_config_types['kconfig-file'].str.contains('/um/')]
df_config_types = df_config_types.merge(df_architectures[['revision', 'committer_date']].drop_duplicates())

df_kconfig = read_dataframe('kconfig')
df_kconfig['year'] = df_kconfig['committer_date'].apply(lambda d: int(d.year))

df_uvl = read_dataframe('model_to_uvl_featureide')
df_smt = read_dataframe('model_to_smt_z3')
df_dimacs = read_dataframe('dimacs')
df_backbone_dimacs = read_dataframe('backbone-dimacs')

df_solve = read_dataframe('solve_model-count', {'model-count': 'string'})
process_model_count(df_solve)

if os.path.isfile(f'{output_directory}/model-count-with-6h-timeout.csv'):
    df_solve_6h = pd.read_csv(f'{output_directory}/model-count-with-6h-timeout.csv', dtype={'model-count': 'string'})
    df_solve_6h = df_backbone_dimacs.merge(df_solve_6h)
    process_model_count(df_solve_6h)
    df_solve = pd.merge(df_solve, df_solve_6h[['revision','architecture', 'extractor', 'backbone.dimacs-analyzer']], indicator=True, how='outer') \
        .query('_merge=="left_only"') \
        .drop('_merge', axis=1)
    df_solve = pd.concat([df_solve, df_solve_6h])
else:
    df_solve_6h = None

for df in [df_kconfig, df_uvl, df_smt, df_dimacs, df_backbone_dimacs, df_solve]:
    replace_values(df)

In [ ]:
# helper functions for drawing plots

def estimate_group(group):
    print('\\hspace{2mm} ' + group + ' \\\\')

def estimate_trend(fig, color=None, color_value=None, xs=[], key=lambda x: x.timestamp()):
    results = px.get_trendline_results(fig)
    if color is not None and color_value is not None:
        idx = [i for i, r in enumerate(results.iloc) if r[color] == color_value][0]
    else:
        idx = 0
    intercept = results.iloc[idx]['px_fit_results'].params[0]
    slope = results.iloc[idx]['px_fit_results'].params[1]
    daily = slope * pd.to_timedelta(1, unit='D').total_seconds()
    weekly = slope * pd.to_timedelta(7, unit='D').total_seconds()
    monthly = slope * pd.to_timedelta(1, unit='D').total_seconds() * 30.437
    yearly = slope * pd.to_timedelta(1, unit='D').total_seconds() * 365.25
    return daily, weekly, monthly, yearly, [intercept + slope * key(x) for x in xs]

def committer_date_x_axis(fig, df=df_kconfig, append_revision=True, step=1):
    axis = df_kconfig[['committer_date', 'revision']].drop_duplicates()
    axis['year'] = axis['committer_date'].apply(lambda d: str(d.year))
    axis = axis.sort_values(by='committer_date').groupby('year').nth(0).reset_index()
    fig.update_xaxes(
        ticktext=axis['year'].str.cat('<br><sup>' + axis['revision'].str[1:] + '</sup>')[1::step] if append_revision else axis['year'][::step],
        tickvals=axis['year'][1::step]
    )

def revision_x_axis(fig, df=df_kconfig):
    axis = df_kconfig[['committer_date', 'revision']].drop_duplicates()
    axis['year'] = axis['committer_date'].apply(lambda d: str(d.year))
    axis = axis.sort_values(by='committer_date').groupby('year').nth(0).reset_index()
    fig.update_xaxes(
        ticktext=axis['year'],
        tickvals=axis['revision']
    )

def log10_y_axis(fig):
    fig.update_yaxes(tickprefix = '10<sup>', ticksuffix = '</sup>')

def percentage_y_axis(fig):
    fig.layout.yaxis.tickformat = ',.0%'

def format_percentage(value):
    return str(round(value * 100, 2)) + '%'

def committer_date_labels(dict={}):
    return {'committer_date': 'Year<br><sup>First Release in Year</sup>'} | dict

def revision_labels(dict={}):
    return {'revision': 'Year'} | dict

def style_legend(fig, position='topleft', xshift=0, yshift=0):
    if position == 'topleft':
        fig.update_layout(legend=dict(yanchor='top', y=0.98 + yshift, xanchor='left', x=0.01 + xshift))
    elif position == 'topright':
        fig.update_layout(legend=dict(yanchor='top', y=0.98 + yshift, xanchor='right', x=0.98 + xshift))
    elif position == 'bottomright':
        fig.update_layout(legend=dict(yanchor='bottom', y=0.01 + yshift, xanchor='right', x=0.98 + xshift))
    elif position == 'bottomleft':
        fig.update_layout(legend=dict(yanchor='bottom', y=0.01 + yshift, xanchor='left', x=0.01 + xshift))
    elif position == 'right':
        return
    elif position == 'horizontal':
        fig.update_layout(legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))
    else:
        fig.update_layout(showlegend=False)

def style_box(fig, legend_position='topleft', xshift=0, yshift=0):
    fig.update_traces(fillcolor='rgba(0,0,0,0)')
    fig.update_traces(line_width=1)
    fig.update_traces(marker_size=2)
    fig.update_layout(font_family="Linux Biolinum")
    style_legend(fig, legend_position, xshift, yshift)

def style_scatter(fig, marker_size=4, legend_position='topleft', xshift=0, yshift=0):
    if marker_size:
        fig.update_traces(marker_size=marker_size)
    style_legend(fig, legend_position, xshift, yshift)
    fig.update_layout(font_family="Linux Biolinum")

def plot_failures(fig, df, x, y, y_value, align='bottom', xref='x', font_size=10, textangle=270):
    group = df.groupby(x, dropna=False)
    failures = (group[y].size() - group[y].count()).reset_index().rename(columns={y: f'{y}_failures'})
    attempts = group[y].size().reset_index().rename(columns={y: f'{y}_attempts'})
    failures = pd.merge(failures, attempts)
    failures[f'{y}_text'] = failures[f'{y}_failures'].astype(str) + ' (' + (failures[f'{y}_failures'] / failures[f'{y}_attempts']).apply(lambda v: "{0:.1f}%".format(v * 100)) + ')'
    for row in range(len(failures)):
        text = failures.at[row, f'{y}_text']
        text = "" if failures.at[row, f'{y}_failures'] == 0 else text
        fig.add_annotation(
            x=failures.at[row, x],
            y=y_value,
            text=text,
            showarrow=False,
            font_size=font_size,
            textangle=textangle,
            align='left' if align == 'bottom' else 'right',
            yanchor='bottom' if align == 'bottom' else 'top',
            yshift=5 if align == 'bottom' else -5,
            font_color='gray',
            xref=xref
        )

def cohens_d(d1, d2):
    # uses pooled standard deviation
    n1, n2 = len(d1), len(d2)
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    u1, u2 = np.mean(d1), np.mean(d2)
    return (u1 - u2) / s

def effect_size_mannwhitneyu(d1, d2):
    u = scipy.stats.mannwhitneyu(d1, d2).statistic
    n1, n2 = len(d1), len(d2)
    n = n1 + n2
    mean_U = n1 * n2 / 2
    std_U = np.sqrt(n1 * n2 * (n + 1) / 12)
    z = (u - mean_U) / std_U
    r = z / np.sqrt(n)
    return r

def wilcoxon_test(df, column_a, column_b):
    # if the same values are returned for many inputs, refer to https://stats.stackexchange.com/q/232927
    a = df[column_a][~df[column_a].isna()]
    b = df[column_b][~df[column_b].isna()]
    d = a - b
    results = scipy.stats.wilcoxon(d, method='approx')
    p = results.pvalue
    # adapted from https://stats.stackexchange.com/q/133077
    r = np.abs(results.zstatistic / np.sqrt(len(d) * 2))
    return p, r

def style_p_values(fig, brackets, scale=0, _format=dict(interline=0.07, text_height=1.07, color='gray')):
    # adapted from https://stackoverflow.com/q/67505252
    for entry in brackets:
        first_column, second_column, y, results = entry
        y_range = [1.01+y*_format['interline'], 1.02+y*_format['interline']]
        p, r = results
        if p >= 0.05:
            symbol = 'ns'
        elif p >= 0.01: 
            symbol = '*'
        elif p >= 0.001:
            symbol = '**'
        else:
            symbol = '***'
        first_column = first_column - scale
        second_column = second_column + scale
        fig.add_shape(type="line",
            xref="x", yref="y domain",
            x0=first_column, y0=y_range[0],
            x1=first_column, y1=y_range[1],
            line=dict(color=_format['color'], width=2,)
        )
        fig.add_shape(type="line",
            xref="x", yref="y domain",
            x0=first_column, y0=y_range[1], 
            x1=second_column, y1=y_range[1],
            line=dict(color=_format['color'], width=2,)
        )
        fig.add_shape(type="line",
            xref="x", yref="y domain",
            x0=second_column, y0=y_range[0], 
            x1=second_column, y1=y_range[1],
            line=dict(color=_format['color'], width=2,)
        )
        fig.add_annotation(dict(font=dict(color=_format['color'],size=14),
            x=(first_column + second_column)/2,
            y=y_range[1]*_format['text_height'],
            showarrow=False,
            text=symbol + ' <sup>(' + str(round(r, 2)) + ')</sup>',
            textangle=0,
            xref="x",
            yref="y domain"
        ))
    return fig

def bracket_for(i, j, xshift, y, results):
    return [i + xshift, j + xshift, y, results]

def filter_extractor(df, extractor):
    return df[df['extractor'] == extractor]

def annotate_value(fig, x, y, subplot, prefix, ax, ay, xanchor, df, fn=lambda prefix, label: prefix + ': ' + label if label else prefix, label=None, size=None):
    if isinstance(x, str):
        x = df[x].iat[0]
    if isinstance(y, str):
        y = log10(df[y].iat[0]/1000000000)
    if isinstance(label, str):
        label = df[label].iat[0]
    else:
        label = format(round(y), ',')  if y > 0 else None
    fig.add_annotation(
        xref='x' + str(subplot),
        yref='y' + str(subplot),
        x=x,
        y=y,
        ax=ax,
        ay=ay,
        xanchor=xanchor,
        text=fn(prefix, label),
        font=dict(size=size)
    )

def show(fig, name=None, width=1000, height=500, margin=None, show=True, format='pdf', scale=1):
    fig.update_layout(width=width, height=height)
    if margin:
        fig.update_layout(margin=margin)
    else:
        fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
    if figures_directory and os.path.isdir(figures_directory) and name:
        fig.write_image(f'{figures_directory}/{name}.{format}', scale=scale)
    if show:
        fig.show()

In [ ]:
# differentiate kinds of features

potential_misses_grep = set()
potential_misses_kmax = set()
extractor_comparison = {}
df_configs_configurable = df_configs.copy()
df_configs_configurable['configurable'] = False

def jaccard(a, b):
    return len(set.intersection(a, b)) / len(set.union(a, b))

def add_features(descriptor, source, features, min=2):
    descriptor[f'#{source}'] = len(features) if features is not None and len(features) >= min else np.nan

def get_variables(variable_map):
    variables = set(variable_map.values())
    if len(variables) <= 1:
        variables = set()
    return variables

def read_unconstrained_feature_variables(extractor, revision, architecture):
    unconstrained_features_filename = f'{output_directory}/unconstrained-features/{extractor}/linux/{revision}[{architecture}].unconstrained.features'
    unconstrained_feature_variables = set()
    if os.path.isfile(unconstrained_features_filename):
        with open(unconstrained_features_filename, 'r') as f:
            unconstrained_feature_variables = set([re.sub('^CONFIG_', '', f.strip()) for f in f.readlines()])
    return unconstrained_feature_variables

def inspect_architecture_features_for_model(extractor, revision, architecture, config_features, features_for_last_revision):
    global potential_misses_grep, potential_misses_kmax
    
    features_filename = f'{output_directory}/kconfig/{extractor}/linux/{revision}[{architecture}].features'
    with open(features_filename, 'r') as f:
        extracted_features = set([re.sub('^CONFIG_', '', f.strip()) for f in f.readlines()])
    
    unconstrained_feature_variables = read_unconstrained_feature_variables(extractor, revision, architecture)

    dimacs_filename = f'{output_directory}/backbone-dimacs/{extractor}/linux/{revision}[{architecture}].backbone.dimacs'
    all_variables = set()
    variables = set()
    feature_variables = set()
    core_feature_variables = set()
    dead_feature_variables = set()
    undead_feature_variables = set()
    all_feature_variables = set()
    features = set()
    core_features = set()
    unconstrained_features = set()
    constrained_features = set()
    added_features = None
    removed_features = None
    infos = {'extracted_features_jaccard': np.nan, \
                     'all_variables_jaccard': np.nan, \
                     'variables_jaccard': np.nan, \
                     'feature_variables_jaccard': np.nan, \
                     'undead_feature_variables_jaccard': np.nan, \
                     'all_feature_variables_jaccard': np.nan, \
                     'features_jaccard': np.nan, \
                     'unconstrained_bools': np.nan, \
                     'unconstrained_tristates': np.nan}
    
    if os.path.isfile(dimacs_filename):
        with open(dimacs_filename, 'r') as f:
            lines = f.readlines()
            all_variable_map = {}
            variable_map = {}
            feature_variable_map = {}
            for f in lines:
                if f.startswith('c '):
                    result = re.search('^c ([^ ]+) ([^ ]+)$', f)
                    if result:
                        index = int(result.group(1).strip())
                        name = result.group(2).strip()
                        all_variable_map[index] = name
                        if "k!" not in name:
                            variable_map[index] = name
                            if name != 'True' \
                                and name != '<unsupported>' \
                                and name != 'PREDICATE_Compare' \
                                and not name.startswith('__VISIBILITY__CONFIG_') \
                                and not name.endswith('_MODULE'):
                                feature_variable_map[index] = name
            all_variables = get_variables(all_variable_map)
            variables = get_variables(variable_map)
            feature_variables = get_variables(feature_variable_map)

            backbone_features_filename = f'{output_directory}/backbone-features/{extractor}/linux/{revision}[{architecture}].backbone.features'
            if os.path.isfile(backbone_features_filename):
                with open(backbone_features_filename, 'r') as f:
                    lines = f.readlines()
                    if len(lines) > 1:
                        core_feature_variables = set([line[1:].strip() for line in lines if line.startswith('+')]).intersection(feature_variables)
                        dead_feature_variables = set([line[1:].strip() for line in lines if line.startswith('-')]).intersection(feature_variables)

            if len(feature_variables) > 0:
                undead_feature_variables = feature_variables.difference(dead_feature_variables)
                all_feature_variables = undead_feature_variables.union(unconstrained_feature_variables)
                features = all_feature_variables.intersection(config_features)
                if f'{revision}###{architecture}' not in extractor_comparison:
                    extractor_comparison[f'{revision}###{architecture}'] = features
                else:
                    extractor_comparison[f'{revision}###{architecture}'] = jaccard(extractor_comparison[f'{revision}###{architecture}'], features)
                core_features = features.intersection(core_feature_variables)
                unconstrained_features = features.intersection(unconstrained_feature_variables)
                unconstrained_features_by_type = pd.DataFrame(list(unconstrained_features), columns=['config']) \
                    .merge(df_config_types[(df_config_types['revision'] == revision)])
                unconstrained_bools = unconstrained_features_by_type[unconstrained_features_by_type['type'] == 'bool']['config'].drop_duplicates()
                unconstrained_tristates = unconstrained_features_by_type[unconstrained_features_by_type['type'] == 'tristate']['config'].drop_duplicates()
                constrained_features = features.difference(core_feature_variables).difference(unconstrained_feature_variables)
                if architecture in features_for_last_revision and len(features_for_last_revision[architecture]) > 0:
                    added_features = features.difference(features_for_last_revision[architecture])
                    removed_features = features_for_last_revision[architecture].difference(features)
                infos = { \
                            'extracted_features_jaccard': jaccard(extracted_features, features), \
                            'all_variables_jaccard': jaccard(all_variables, features), \
                            'variables_jaccard': jaccard(variables, features), \
                            'feature_variables_jaccard': jaccard(feature_variables, features), \
                            'undead_feature_variables_jaccard': jaccard(undead_feature_variables, features), \
                            'all_feature_variables_jaccard': jaccard(all_feature_variables, features), \
                            'features_jaccard': 1, \
                            'unconstrained_bools': len(unconstrained_bools), \
                            'unconstrained_tristates': len(unconstrained_tristates) \
                        }
    descriptor = {'extractor': extractor, 'revision': revision, 'architecture': architecture} | infos
    add_features(descriptor, 'config_features', config_features) # F_config
    add_features(descriptor, 'extracted_features', extracted_features) # F_extracted
    add_features(descriptor, 'unconstrained_feature_variables', unconstrained_feature_variables, min=1) # F_unconstrained
    add_features(descriptor, 'all_variables', all_variables) # V_all
    add_features(descriptor, 'variables', variables) # V_phi
    add_features(descriptor, 'feature_variables', feature_variables) # FV_phi
    add_features(descriptor, 'core_feature_variables', core_feature_variables, min=1) # FV_core
    add_features(descriptor, 'dead_feature_variables', dead_feature_variables, min=1) # FV_dead
    add_features(descriptor, 'constrained_feature_variables', undead_feature_variables.difference(core_feature_variables)) # FV_constrained
    add_features(descriptor, 'undead_feature_variables', undead_feature_variables) # FV_undead
    add_features(descriptor, 'all_feature_variables', all_feature_variables) # FV
    add_features(descriptor, 'ALL_feature_variables', feature_variables.union(unconstrained_feature_variables)) # FV_all
    add_features(descriptor, 'features', features) # F
    add_features(descriptor, 'core_features', core_features, min=1)
    add_features(descriptor, 'unconstrained_features', unconstrained_features, min=1)
    add_features(descriptor, 'constrained_features', constrained_features)
    add_features(descriptor, 'added_features', added_features, min=0)
    add_features(descriptor, 'removed_features', removed_features, min=0)
    if extractor == 'kmax':
        potential_misses_grep.update([f for f in all_feature_variables.difference(features) if '__CONFIG_' not in f])
    return descriptor, feature_variables.union(unconstrained_feature_variables), features

def inspect_architecture_features_for_revision(extractor, revision, features_for_last_revision):
    config_features = set(df_configs[df_configs['revision'] == revision]['config'])
    architectures = [re.search('\[(.*)\]', f).group(1) for f in glob.glob(f'{output_directory}/kconfig/{extractor}/linux/{revision}[*.features')]
    architectures = list(set(architectures))
    architectures.sort()
    data = []
    total_features = set()
    total_feature_variables = set()
    features_for_current_revision = {}
    for architecture in architectures:
        descriptor, feature_variables, features = inspect_architecture_features_for_model(extractor, revision, architecture, config_features, features_for_last_revision)
        data.append(descriptor)
        total_features.update(features)
        features_for_current_revision[architecture] = features
        if extractor == 'kmax':
            total_feature_variables.update(feature_variables)
    for descriptor in data:
        add_features(descriptor, 'total_features', total_features)
        total_added_features = None
        total_removed_features = None
        if 'TOTAL' in features_for_last_revision and len(features_for_last_revision['TOTAL']) > 0:
            total_added_features = total_features.difference(features_for_last_revision['TOTAL'])
            total_removed_features = features_for_last_revision['TOTAL'].difference(total_features)
        add_features(descriptor, 'total_added_features', total_added_features, min=0)
        add_features(descriptor, 'total_removed_features', total_removed_features, min=0)
    features_for_current_revision['TOTAL'] = total_features
    df_configs_configurable.loc[(df_configs_configurable['revision'] == revision) & (df_configs_configurable['config'].isin(total_features)), 'configurable'] = True
    if extractor == 'kmax':
        potential_misses_kmax.update([f for f in config_features.difference(total_feature_variables)])
    return data, features_for_current_revision

def inspect_architecture_features(extractor):
    print(f'{extractor} ', end='')
    revisions = [re.search('/linux/(.*)\[', f).group(1) for f in glob.glob(f'{output_directory}/kconfig/{extractor}/linux/*.features')]
    revisions = list(set(revisions))
    revisions.sort(key=Version)
    data = []
    features_for_last_revision = {}
    i = 0
    for revision in revisions:
        i += 1
        if i % 10 == 0:
            print(revision + ' . ', end='')
        new_data, features_for_last_revision = inspect_architecture_features_for_revision(extractor, revision, features_for_last_revision)
        data += new_data
    print()
    return data

if os.path.isfile(f'{output_directory}/linux-features.dat'):
    with open(f'{output_directory}/linux-features.dat', 'rb') as f:
        [features_by_kind_per_architecture, df_extractor_comparison, potential_misses_grep, potential_misses_kmax, df_configs_configurable] = pickle.load(f)
else:
    features_by_kind_per_architecture = inspect_architecture_features('kconfigreader')
    features_by_kind_per_architecture += inspect_architecture_features('kmax')
    features_by_kind_per_architecture = pd.DataFrame(features_by_kind_per_architecture)
    df_extractor_comparison = []
    for key, value in extractor_comparison.items():
        [revision, architecture] = key.split('###')
        if type(value) is set:
            value = pd.NA
        df_extractor_comparison.append({'revision': revision, 'architecture': architecture, 'extractor_jaccard': value})
    df_extractor_comparison = pd.DataFrame(df_extractor_comparison)
    with open(f'{output_directory}/linux-features.dat', 'wb') as f:
        pickle.dump([features_by_kind_per_architecture, df_extractor_comparison, potential_misses_grep, potential_misses_kmax, df_configs_configurable], f)

replace_values(features_by_kind_per_architecture)
df_features = pd.merge(df_architectures, features_by_kind_per_architecture, how='outer').sort_values(by='committer_date')
df_features = pd.merge(df_kconfig, df_features, how='outer').sort_values(by='committer_date')

def compare_with_grep(message, list):
    print(f'{message}: ' + str(len(list)))
    print(pd.merge(df_configs[['config','kconfig-file']], pd.DataFrame(list, columns=['config']), how='inner') \
        .drop_duplicates().merge(df_config_types[['config', 'type']]).drop_duplicates())

def report_potential_misses(potential_misses_grep, potential_misses_kmax):
    # these are the features NOT found by grep, but found by kmax (this allows us to check whether the grep regex matches too much)
    # the only matches are enviroment variables (e.g., ARCH) and mistakes in kconfig files: IA64_SGI_UV (which has a trailing `) and SND_SOC_UX500_MACH_MOP500 (which has a leading +)
    compare_with_grep('#potential misses (grep)', potential_misses_grep)
    print()

    # these are the features found by grep, but NOT found by kmax, either constrained or unconstrained (this allows us to check whether kmax matches enough)
    # as there are some extraction failures for kmax, we expect some misses; also, we do not extract the um architecture; and finally, there are some test kconfig files that are never included
    # in the following, we try to filter out these effects (this is not perfect though)
    potential_misses_kmax_with_type = (pd.merge(df_configs[['config','kconfig-file', 'revision']], pd.DataFrame(potential_misses_kmax, columns=['config']), how='inner') \
            .drop_duplicates().merge(df_config_types[['config', 'type']]).drop_duplicates())
    misses_due_to_tests = set(potential_misses_kmax_with_type[ \
            potential_misses_kmax_with_type['kconfig-file'].str.startswith('Documentation/') | \
            potential_misses_kmax_with_type['kconfig-file'].str.startswith('scripts/')]['config'].unique())
    missing_kmax_models = df_features[(df_features['extractor'] == 'KClause') & df_features['#extracted_features'].isna()]
    missing_kmax_models = missing_kmax_models[['revision', 'architecture']].drop_duplicates()
    potential_misses_kmax_with_type['architecture'] = potential_misses_kmax_with_type['kconfig-file'].apply(lambda s: re.sub(r'^arch/(.*?)/.*$', r'\1', s))
    potential_misses_due_to_missing_kmax_models = set(potential_misses_kmax_with_type.merge(missing_kmax_models[['revision', 'architecture']].drop_duplicates()) \
                                                    .drop(columns=['kconfig-file', 'revision', 'architecture', 'type'])['config'].unique())
    potential_misses_kmax = potential_misses_kmax.difference(misses_due_to_tests).difference(potential_misses_due_to_missing_kmax_models)
    # the remaining matches are due to our way of using kmax extractor, where we ignore lines with new kconfig constructs like $(success,...)
    compare_with_grep('#potential misses (kmax)', potential_misses_kmax)

report_potential_misses(potential_misses_grep, potential_misses_kmax)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

df_total_features = df_features.groupby(['extractor', 'revision']).agg({'#total_features': 'min'}).reset_index()
df_total_features = pd.merge(df_kconfig[['committer_date', 'revision']].drop_duplicates(), df_total_features)
df_total_features = df_total_features[df_total_features['extractor']=='KClause']
df = df_total_features.sort_values(by='committer_date')
df = df[['committer_date', '#total_features']].drop_duplicates()
fig.add_trace(
    go.Scatter(x=df['committer_date'], y=df['#total_features'], mode='markers', name='Number of Features (r=.99)', marker_symbol='circle'),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(x=df_kconfig['committer_date'], y=df_kconfig['source_lines_of_code'], mode='markers', name='Source Lines of Code (r=.98)', marker_symbol='circle'),
    secondary_y=True
)

fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="Number of Features", dtick=5000, range=[0, 22000], color="#648FFF", title_font_color="black", secondary_y=False)
fig.update_yaxes(title_text="Source Lines of Code", dtick=10000000, range=[0, 32000000], color="#FE6100", title_font_color="black", secondary_y=True)

style_scatter(fig, marker_size=3, legend_position='topleft')
fig.update_xaxes(range=["2002-01-01", "2025-01-01"])
show(fig, 'sloc', width=400, height=260)

In [ ]:
# statistics in section 1

# show(px.scatter(
#     df_kconfig,
#     x='committer_date',
#     y='source_lines_of_code',
#     trendline='ols'
# ))

# show(px.scatter(
#     df,
#     x='committer_date',
#     y='#total_features',
#     trendline='ols'
# ))

# linear regression of source lines of code and number of features
print(scipy.stats.pearsonr(df_kconfig['committer_date'].astype(int) // 10 ** 9, df_kconfig['source_lines_of_code']))
print(scipy.stats.pearsonr(df['committer_date'].astype(int) // 10 ** 9, df['#total_features']))
print(scipy.stats.linregress(df_kconfig['committer_date'].astype(int) // 10 ** 9, df_kconfig['source_lines_of_code']))
print(scipy.stats.linregress(df['committer_date'].astype(int) // 10 ** 9, df['#total_features']))

# lines of code per year
print(0.032409366566693104*60*60*24*365.25)
# features per year
print(2.6766885381596744e-05*60*60*24*365.25)


In [ ]:
def get_year(d):
    try:
        return 2000 + int(d.split('-')[0])
    except:
        if d == 'IsaSAT' or d == 'MergeSat':
            return 2023
        return int(d.split('-')[-1])

def scope_to(df, source_descriptor):
    if 'extra' not in source_descriptor:
        df = df[df['source-analyzer']!='extra']
    if 'extra' in source_descriptor and not source_descriptor.endswith('+timeout'):
        # most timeouts are caused by the stochastic local search solvers (SLS) unitwalk and adaptg2wsat2011 on unsatisfiable instances (see below)
        # these solvers are not able to solve/prove unsatisfiable instances, so it is unfair to include them in the comparison
        # thus, we exclude their timeouts here, as the results are necessarily confounded by the number of unsatisfiable queries in the sample
        df = df[~df['model-satisfiable'].isna()|(~df['dimacs-analyzer'].str.contains('unitwalk')&~df['dimacs-analyzer'].str.contains('adaptg2wsat2011'))]
    if source_descriptor.endswith('+timeout'):
        source_descriptor = source_descriptor[:-len('+timeout')]
    if source_descriptor.endswith('==2024'):
        df = df[df['year-analyzer']==2024]
        source_descriptor = source_descriptor[:-len('==2024')]
    if source_descriptor.endswith('!=2024'):
        df = df[df['year-analyzer']!=2024]
        source_descriptor = source_descriptor[:-len('!=2024')]
    if source_descriptor.endswith('==2023'):
        df = df[df['year-analyzer']==2023]
        source_descriptor = source_descriptor[:-len('==2023')]
    if source_descriptor.endswith('!=2023'):
        df = df[df['year-analyzer']!=2023]
        source_descriptor = source_descriptor[:-len('!=2023')]
    if len(source_descriptor) > 0:
        if source_descriptor == 'sat-museum+competition':
            return df[(df['source-analyzer']=='sat-museum')|((df['source-analyzer']=='sat-competition')&(df['year-analyzer']>=2023))]
        df = df[df['source-analyzer']==source_descriptor]
    return df

pd.set_option('mode.copy_on_write', True)
output_directory = '../../output-archive/output-fin-2025-06-19'
df2_sat = read_dataframe(f'solve_model-satisfiable')
df2_dimacs = read_dataframe(f'dimacs')
output_directory = '../../output-archive/stages-time-travel-2025-11-02'
df2_sat_extra = read_dataframe(f'solve_sat')
df2_solver_loc = read_dataframe(f'solver_loc')
df2_dimacs['year'] = df2_dimacs['committer_date'].apply(lambda d: int(d.year))
for df in [df2_sat, df2_sat_extra, df2_dimacs]:
    replace_values(df)
    df.replace('i386', 'x86', inplace=True)
    df.replace('model_to_dimacs_kconfigreader', 'KConfigReader', inplace=True)
    df.replace('smt_to_dimacs_z3', 'Z3', inplace=True)
    df.replace('sat-competition/', '', inplace=True, regex=True)
    df.replace('/home/input/sat_heritage/sat-heritage/run', '', inplace=True, regex=True)
    df.replace('other/', '', inplace=True, regex=True)
    df.replace('.sh', '', inplace=True, regex=True)
    df.replace('SAT4J.210', '09-Sat4j 2.1.0', inplace=True, regex=True)
    df.replace('SAT4J.231', '11-Sat4j 2.3.1', inplace=True, regex=True)
    df.replace('SAT4J.235', '14-Sat4j 2.3.5', inplace=True, regex=True)
df2_sat['query'] = 'void'
df2_sat_extra = df2_sat_extra.rename(columns={'dimacs_file': 'dimacs-file', 'dimacs_query': 'query', 'dimacs_solver': 'dimacs-analyzer', 'dimacs_solver_time': 'dimacs-analyzer-time', 'sat': 'model-satisfiable'})
df2_sat_extra['dimacs-file'] = 'smt_to_dimacs_z3/kmax/1/' + df2_sat_extra['dimacs-file'].astype(str)
df2_sat_extra['iteration'] = 1
df2_sat_extra['dimacs-analyzer'] = df2_sat_extra['dimacs-analyzer'].str.strip()
df2_sat['year-analyzer'] = df2_sat['dimacs-analyzer'].apply(get_year)
df2_sat_extra['year-analyzer'] = df2_sat_extra['dimacs-analyzer'].apply(get_year)
df2_sat['source-analyzer'] = df2_sat['dimacs-analyzer'].str.split('-', expand=True)[0].str.isdigit().map({True: 'sat-competition', False: 'sat-museum'})
df2_sat_extra['source-analyzer'] = 'extra'
df2_sat.loc[df2_sat['dimacs-analyzer'].str.contains('Sat4j'), 'source-analyzer'] = 'FeatureIDE'
df2_sat = pd.concat([df2_sat, df2_sat_extra])
df2=df2_sat.merge(df2_dimacs).merge(df_features, on=['model-file'], how='left', suffixes=(None, '_2'))
df2_solver_loc.rename(columns={'dimacs_solver': 'dimacs-analyzer', 'source_lines_of_code': 'solver_source_lines_of_code', 'source_lines_of_code_extended': 'solver_source_lines_of_code_extended', 'language': 'solver_language'}, inplace=True)
df2_solver_loc['solver_source_lines_of_code_extended'] = df2_solver_loc['solver_source_lines_of_code_extended'].fillna(df2_solver_loc['solver_source_lines_of_code'])
df2=df2.merge(df2_solver_loc.rename(columns={'dimacs_solver': 'dimacs-analyzer'}), on=['dimacs-analyzer'], how='left')
pd.set_option('mode.copy_on_write', False)
print('unexpectedly unsatisfiable instances: ' + str(len(df2[(df2['model-satisfiable']==False)&(df2['query']=='void')])))
df2

In [ ]:
# here we determine whether there is a noticeable influence of iterations in our experiment
# there isn't for all solvers except 24-kissat-sc2024, so in the following we only plot median values

df_iterations = []
# for source in ['==2024', 'sat-competition!=2024', 'sat-museum', 'FeatureIDE']:
for source in ['==2024', '!=2024']:
    pd.set_option('mode.copy_on_write', True)
    df_tmp = scope_to(df2.copy(), source)
    # df_tmp = df_tmp[df_tmp['model-file'].str.startswith('kconfigreader/1')|df_tmp['model-file'].str.startswith('kmax/1')] # both extractors significantly affect kissat
    # df_tmp = df_tmp[df_tmp['dimacs-file'].str.startswith('model_to_dimacs_kconfigreader/1')] # the KConfigReader transformation significantly affects kissat
    df_tmp['dimacs-analyzer-time-normalized'] = df_tmp[['year', 'revision', 'architecture', 'extractor', 'dimacs-transformer', 'dimacs-analyzer', 'dimacs-analyzer-time']].groupby(['year', 'revision', 'architecture', 'extractor', 'dimacs-transformer', 'dimacs-analyzer']).transform(lambda x: (x / x.median()))
    df_tmp['dimacs-analyzer-time-z-score'] = df_tmp[['year', 'revision', 'architecture', 'extractor', 'dimacs-transformer', 'dimacs-analyzer', 'dimacs-analyzer-time']].groupby(['year', 'revision', 'architecture', 'extractor', 'dimacs-transformer', 'dimacs-analyzer']).transform(lambda x: (x - x.mean()) / x.std())
    df_tmp['iteration-source'] = source
    df_iterations.append(df_tmp)
    pd.set_option('mode.copy_on_write', False)
df_iterations = pd.concat(df_iterations)

# for y in ['dimacs-analyzer-time-normalized', 'dimacs-analyzer-time-z-score']:
for y in ['dimacs-analyzer-time-normalized']:
    fig = px.box(
    # fig = px.violin(
        df_iterations,
        # points=False,
        # x=df_tmp['year-analyzer'],
        facet_col='iteration-source',
        color='iteration-source',
        y=df_iterations[y],
        # Rel. Dev. Median SAT Runtime
        labels={'dimacs-analyzer-time-normalized': 'Relative Deviation (log<sub>10</sub>)', 'dimacs-analyzer-time-z-score': 'z-Score For SAT Solving Time', 'architecture': 'Architecture', 'dimacs-analyzer': 'SAT Solver'},
        category_orders={'iteration-source': ['==2024', '!=2024']},
        hover_data=['revision', 'dimacs-analyzer', 'architecture', 'dimacs-transformer', 'extractor'],
        log_y=True,
        # points=False
    )
    fig.update_layout(boxmode='group', boxgap=0.2)
    fig.update_traces(width=0.02)
    style_box(fig, legend_position=None)
    show(fig, 'factor-iteration', width=100, height=0.6*default_height, format='png', scale=4)

In [ ]:
for factor in ['architecture', 'extractor', 'dimacs-transformer', 'source-analyzer']:
    print(factor)
    df_factor = scope_to(df2, '')
    if factor == 'architecture':
        category_orders = ['x86', 'arm']
    elif factor == 'extractor':
        category_orders = ['KConfigReader', 'KClause']
    elif factor == 'dimacs-transformer':
        category_orders = ['KConfigReader', 'Z3']
    elif factor == 'source-analyzer':
        category_orders = ['sat-competition', 'sat-museum']
        # does the gcc compiler version significantly affect results?
        df_factor = df_factor[(df_factor['source-analyzer']=='sat-competition') | (df_factor['source-analyzer']=='sat-museum')]
        # 2002 and 2003 are different solvers in both datasets
        # 2023 and 2024 are not included in the museum dataset
        df_factor = df_factor[(df_factor['year-analyzer']!=2002)&(df_factor['year-analyzer']!=2003)&(df_factor['year-analyzer']!=2023)&(df_factor['year-analyzer']!=2024)]

    fig = px.box(
        df_factor,
        y=df_factor['dimacs-analyzer-time'] / 1000000000,
        facet_col=factor,
        color=factor,
        log_y=True,
        category_orders={factor: category_orders},
        labels={'y': 'SAT Runtime (log<sub>10</sub> s)'},
        boxmode='group',
        # points=False
    )
    fig.update_layout(boxmode='group', boxgap=0.2)
    fig.update_traces(width=0.02)
    style_box(fig, legend_position=None)
    show(fig, f'factor-{factor}', width=80, height=0.6*default_height, margin=dict(l=40, r=0, t=0, b=0), format='png', scale=4)
    # print(scipy.stats.ttest_ind(*df_factor.groupby(factor)['dimacs-analyzer-time'].apply(lambda x:x.values)))
    # print(cohens_d(*df_factor.groupby(factor)['dimacs-analyzer-time'].apply(lambda x:x.values)))
    print(scipy.stats.mannwhitneyu(*df_factor.groupby(factor)['dimacs-analyzer-time'].apply(lambda x:x.values)))
    print(effect_size_mannwhitneyu(*df_factor.groupby(factor)['dimacs-analyzer-time'].apply(lambda x:x.values)))
    if factor == 'dimacs-transformer':
        df_factor_dimacs_transformer = []
        fig = px.box(df_factor.replace({'KConfigReader': 'KCR'}), x='dimacs-transformer', color='dimacs-transformer', y=df_factor['dimacs-analyzer-time'] / 1000000000, facet_col='year-analyzer', log_y=True, labels={'dimacs-transformer': '', 'source-analyzer': ''})
        # show(fig, width=4*330, height=default_height)
        for extractor in df_factor['extractor'].unique():
            for solver in df_factor['dimacs-analyzer'].unique():
                df_factor_dimacs_transformer.append({'extractor': extractor, 'dimacs-analyzer': solver, 'significant': scipy.stats.mannwhitneyu(*df_factor[(df_factor['dimacs-analyzer']==solver)&(df_factor['extractor']==extractor)].groupby(factor)['dimacs-analyzer-time'].apply(lambda x:x.values)).pvalue < 0.005, 'effect_size': effect_size_mannwhitneyu(*df_factor[(df_factor['dimacs-analyzer']==solver)&(df_factor['extractor']==extractor)].groupby(factor)['dimacs-analyzer-time'].apply(lambda x:x.values))})
        for extractor in df_factor['extractor'].unique():
            if scipy.stats.mannwhitneyu(*df_factor[(df_factor['extractor']==extractor)].groupby(factor)['dimacs-analyzer-time'].apply(lambda x:x.values)).pvalue < 0.005:
                print(str(extractor) + 'u: ' + str(effect_size_mannwhitneyu(*df_factor[(df_factor['extractor']==extractor)].groupby(factor)['dimacs-analyzer-time'].apply(lambda x:x.values))))
        df_factor_dimacs_transformer = pd.DataFrame(df_factor_dimacs_transformer)
    if factor == 'source-analyzer':
        fig = px.box(df_factor, x='source-analyzer', color='source-analyzer', y=df_factor['dimacs-analyzer-time'] / 1000000000, facet_col='year-analyzer', log_y=True, labels={'dimacs-transformer': '', 'source-analyzer': ''})
        
        # show(fig, width=4*330, height=default_height)
        # the solvers differ and are not comparable in 2002 and 2003, so we omit them here
        for year in range(2004, 2023):
            # if scipy.stats.ttest_ind(*df_factor[(df_factor['year-analyzer']==year)].groupby('source-analyzer')['dimacs-analyzer-time'].apply(lambda x:x.values)).pvalue < 0.005:
            #     print(str(year) + ' t: ' + str(cohens_d(*df_factor[(df_factor['year-analyzer']==year)].groupby('source-analyzer')['dimacs-analyzer-time'].apply(lambda x:x.values))))
            if scipy.stats.mannwhitneyu(*df_factor[(df_factor['year-analyzer']==year)].groupby('source-analyzer')['dimacs-analyzer-time'].apply(lambda x:x.values)).pvalue < 0.005:
                print(str(year) + 'u: ' + str(effect_size_mannwhitneyu(*df_factor[(df_factor['year-analyzer']==year)].groupby('source-analyzer')['dimacs-analyzer-time'].apply(lambda x:x.values))))
    print('variance comparison:')
    variance = df_factor.groupby(factor)['dimacs-analyzer-time'].var()
    print(variance)
    print((variance.iloc[0]/variance.iloc[1])-1)

In [ ]:
# in the median of the entire experiment, CNF transformation has a negligible effect
# however, for many specific extractor-solver combinations, there is a significant (p<.005) effect
# here we plot the effect sizes as a histogram to check whether it matters which extractor or solver we consider
# the result shows that KConfigReader extraction pairs well with Z3 CNF transformation (and, surprisingly, not well with its own integrated CNF transformation)
# and KClause extraction pairs well with KConfigReader CNF transformation (not with Z3, as is done all throughout the KMax tooling)
# the solvers themselves matter much less than the extractor
# this is a new and counterintuitive result, and it remains to be seen whether this holds on other systems than Linux
df_current = df_factor_dimacs_transformer[df_factor_dimacs_transformer['significant']]
bin_width = 0.05 # adjust as needed
max_val = np.ceil(np.max(np.abs(df_current['effect_size'])) / bin_width) * bin_width
start = -max_val - bin_width / 2
end = max_val + bin_width / 2
fig=px.histogram(df_current, x='effect_size', labels={'effect_size': 'Effect Size (Mann-Whitney U)'}, color='extractor')
fig.update_traces(xbins=dict(start=start, end=end, size=bin_width))
fig.update_layout(yaxis_title_text='#Extractor+Solver')
show(fig, width=400, height=200)

In [ ]:
def plot(df, x, legend_position=None, facet_col=None, facet_row=None, color=None, color_discrete_sequence=None, xshift=0, yshift=0, color_value=None, agg='median', remove_architecture=True, extractor_label='Extractor', extractor_labels=['KConfigReader', 'KClause'], architecture_label='Architecture', architecture_labels=['x86', 'arm']):
    df_tmp = df.copy()
    # remove iterations, which almost always have negligible influence
    df_tmp = df_tmp.groupby(['year', 'year-analyzer', 'committer_date', 'revision', 'architecture', 'extractor', 'dimacs-transformer', 'dimacs-analyzer', 'source-analyzer']).agg({'dimacs-analyzer-time': agg}).reset_index()
    # remove second architecture, which it does not differ significantly
    if remove_architecture:
        df_tmp = df_tmp[df_tmp['architecture'] == 'x86']
    # create additional, optional column for easier plotting
    df_tmp['facet'] = df_tmp['extractor'] + ', ' + df_tmp['dimacs-transformer']

    df_stats = None
    if x == 'committer_date':
        fig_ols = px.scatter(
            df_tmp,
            x=df_tmp[x],
            y=df_tmp['dimacs-analyzer-time'] / 1000000000,
            facet_col=facet_col,
            facet_row=facet_row,
            color=color,
            symbol='architecture',
            log_y=True,
            trendline='ols',
            trendline_options=dict(log_y=True)
        )
        results = px.get_trendline_results(fig_ols)
        df_stats = []
        for i, r in enumerate(results.iloc):
            color_value = r[color] if color is not None else None
            facet_col_value = r[facet_col] if facet_col is not None else None
            facet_row_value = r[facet_row] if facet_row is not None else None
            symbol_value = r['architecture']
            slope = results.iloc[i]['px_fit_results'].params[1]
            yearly = (10**(slope * pd.to_timedelta(1, unit='D').total_seconds() * 365.25))-1
            df_row = df_tmp
            if color_value is not None:
                df_row = df_row[df_row[color] == color_value]
            if facet_col_value is not None:
                df_row = df_row[df_row[facet_col] == facet_col_value]
            if facet_row_value is not None:
                df_row = df_row[df_row[facet_row] == facet_row_value]
            if symbol_value is not None:
                df_row = df_row[df_row['architecture'] == symbol_value]
            s = scipy.stats.pearsonr(df_row[x].astype(int) // 10 ** 9, np.log10(df_row['dimacs-analyzer-time'] / 1000000000))
            # print(f'{color_value} {facet_col_value} {facet_row_value} {symbol_value}: {yearly:.2%} {s.statistic:.2f} {s.pvalue:.2f}')
            df_stats.append({'yearly': yearly, 'r': s.statistic, 'p': s.pvalue, 'color': color_value, 'facet_col': facet_col_value, 'facet_row': facet_row_value, 'symbol': symbol_value})
        df_stats = pd.DataFrame(df_stats)

    fig = px.line(
        df_tmp,
        x=df_tmp[x],
        y=df_tmp['dimacs-analyzer-time'] / 1000000000,
        labels={
            'y': 'SAT Runtime (log<sub>10</sub> s)',
            'facet': 'Extractor, Transformation',
            'dimacs-analyzer': 'SAT Solver',
            'architecture': architecture_label,
            'extractor': extractor_label,
            'dimacs-transformer': 'Transformation',
            'year-analyzer': 'Year of SAT Solver',
            'committer_date': 'Year of Feature Model',
            'revision': 'Revision'
        },
        hover_data=['revision', 'dimacs-analyzer'],
        facet_col=facet_col,
        facet_row=facet_row,
        color=color,
        symbol='architecture',
        line_dash='architecture',
        symbol_sequence=['circle', 'circle-open'],
        line_dash_sequence=['solid', 'dot'],
        category_orders={
            'extractor': extractor_labels,
            'dimacs-transformer': ['KConfigReader', 'Z3'],
            'architecture': architecture_labels,
            'facet': ['KConfigReader, KConfigReader', 'KConfigReader, Z3', 'KClause, KConfigReader', 'KClause, Z3'],
        },
        color_discrete_sequence=color_discrete_sequence,
        log_y=True,
        markers=True
    )

    fig.update_traces(line_width=1)
    style_scatter(fig, legend_position=legend_position)
    return fig, df_stats

def generate_gradient():
    gradient = []
    n=8
    for i in range(n):
        value = int(255 * (((n+1) - i) / (n+1)))
        value = int(255 * (i / (n+1)))
        color = f'#{value:02x}{value:02x}{value:02x}'
        gradient.append(color)
    n=10
    for i in range(n):
        value = int(255 * (((n+1) - i) / (n+1)))
        value = int(255 * (i / (n+1)))
        color = f'#00{value:02x}ff'
        gradient.append(color)
    n=5
    for i in range(n):
        value = int(255 * (((n+1) - i) / (n+1)))
        value = int(255 * (i / (n+1)))
        # blue_value = int(255 * (1 - (i / n)))
        # red_value = int(255 * (i / n))
        color = f'#ff{value:02x}00'
        gradient.append(color)
    return gradient

def lreplace(pattern, sub, string):
    return re.sub('^%s' % pattern, sub, string)

def stats_table(df_stats, group_attributes=['color']):
    f = lambda x: lreplace('0\\.', '.', x.replace('_', '\\_').replace('%', '\\%'))
    pf = lambda x: f(f'{x:.3f}')
    rf = lambda x: f(f'{x:.2f}')
    yf = lambda x: f(f'{x:.1%}')
    df = df_stats.copy()
    df = df.assign(noop='')
    if group_attributes is None:
        group_attributes = ['noop']
    df = df.groupby(group_attributes).agg({'p': ['min', 'median', 'max'], 'r': ['min', 'median', 'max'], 'yearly': ['min', 'median', 'max']}).reset_index()
    latex_table = df.to_latex(index=False, formatters=[f, pf, pf, pf, rf, rf, rf, yf, yf, yf])
    print(latex_table)

def annotate_solvers(fig, df, ay=-15):
    fn1 = lambda prefix, label: "'" + str(label)[2:4]
    last_index = None
    last_value = None
    for idx, revision in enumerate(df['revision'].unique()):
        df_tmp = df[df['revision'] == revision]
        current_value = df_tmp['year-analyzer'].iat[0]
        height = 1 if idx % 2 == 0 or last_index < idx - 1 else 1.8
        if current_value != last_value:
            annotate_value(fig, 'committer_date', 'dimacs-analyzer-time', 1, '', 0, ay*height, 'center', df_tmp, fn1, 'year-analyzer', 10)
            last_index = idx
        last_value = current_value

In [ ]:
df_current = scope_to(df2, 'sat-museum+competition')
fig, df_stats = plot(df_current, x='committer_date', facet_col='extractor', facet_row='dimacs-transformer', color='dimacs-analyzer', color_discrete_sequence=generate_gradient(), legend_position='right', remove_architecture=False)
show(fig, width=1500, height=2*default_height, margin=dict(l=0, r=0, t=20, b=0))
fig, _ = plot(df_current, x='committer_date', facet_col='extractor', facet_row='dimacs-transformer', color='dimacs-analyzer', color_discrete_sequence=generate_gradient())
show(fig, 'sat', width=450, height=1.2*default_height, margin=dict(l=0, r=0, t=20, b=0))
# print(scipy.stats.pearsonr(df2['committer_date'].astype(int) // 10 ** 9, df2['source_lines_of_code']))
# px.box(
#     df2,
#     x=df2['committer_date'],
#     y=df2['dimacs-analyzer-time'] / 1000000000,
#     facet_col='extractor',
#     facet_row='dimacs-transformer',
#     log_y=True
# )
stats_table(df_stats)

In [ ]:
df_current = scope_to(df2, 'sat-competition')
fig, df_stats = plot(df_current, x='committer_date', facet_col='extractor', facet_row='dimacs-transformer', color='dimacs-analyzer', color_discrete_sequence=generate_gradient(), legend_position='right', remove_architecture=False)
show(fig, width=1500, height=2*default_height, margin=dict(l=0, r=0, t=20, b=0))
df_current = scope_to(df2, 'sat-museum')
fig, df_stats = plot(df_current, x='committer_date', facet_col='extractor', facet_row='dimacs-transformer', color='dimacs-analyzer', color_discrete_sequence=generate_gradient(), legend_position='right', remove_architecture=False)
show(fig, width=1500, height=2*default_height, margin=dict(l=0, r=0, t=20, b=0))

In [ ]:
df_current = scope_to(df2, 'sat-museum+competition')
fig, _ = plot(df_current[df_current['year-analyzer']==2024], x='committer_date', color='facet', agg='max')
show(fig, width=250, height=0.7*default_height, margin=dict(l=0, r=0, t=20, b=0))
fig, _ = plot(df_current[df_current['year-analyzer']==2024], x='committer_date', color='facet', agg='min')
show(fig, width=250, height=0.7*default_height, margin=dict(l=0, r=0, t=20, b=0))

In [ ]:
df_current = scope_to(df2, 'sat-museum+competition')
fig, _ = plot(df_current, x='year-analyzer', facet_col='extractor', facet_row='dimacs-transformer', color='year', color_discrete_sequence=generate_gradient(), legend_position='right', remove_architecture=False)
show(fig, width=1500, height=2*default_height, margin=dict(l=0, r=0, t=20, b=0))
fig, _ = plot(df_current, x='year-analyzer', facet_col='extractor', facet_row='dimacs-transformer', color='year', color_discrete_sequence=generate_gradient())
# fig.add_vrect(x0="2002.5", x1="2003.6", annotation_text="'03", annotation_position="top left", annotation_textangle=-90, fillcolor="gray", opacity=0.1, line_width=0)
# fig.add_vrect(x0="2022.5", x1="2023.6", annotation_text="'23", annotation_position="top left", annotation_textangle=-90, fillcolor="gray", opacity=0.1, line_width=0)
# fig.add_vrect(x0="2023.5", x1="2024.6", annotation_text="'24", annotation_position="top left", annotation_textangle=-90, fillcolor="gray", opacity=0.1, line_width=0)
show(fig, 'fm', width=460, height=1.2*default_height, margin=dict(l=0, r=10, t=20, b=0))

In [ ]:
df_current = scope_to(df2, 'sat-museum+competition')
fig, df_stats = plot(df_current[df_current['year-analyzer'] == df_current['year']], x='committer_date', color='facet', legend_position='right', remove_architecture=False)
show(fig, width=700, height=default_height, margin=dict(l=0, r=0, t=20, b=0))
df_current = df_current[df_current['year-analyzer'] == df_current['year']]
fig, _ = plot(df_current, x='committer_date', color='facet')
fig.update_yaxes(range=[-2.1, 2.6])
show(fig, 'equals', width=250, height=0.7*default_height, margin=dict(l=0, r=0, t=20, b=0))
stats_table(df_stats, None)
# fig, _ = plot(df_current[df_current['architecture']=='x86'], x='committer_date', color='facet', legend_position='horizontal')
# show(fig, 'legend', width=1500, height=default_height, margin=dict(l=0, r=0, t=20, b=0))

In [ ]:
df_current = scope_to(df2, 'sat-museum+competition')
df3 = df_current[df_current['year-analyzer'] <= df_current['year']]
df3 = df3.loc[df3.groupby(['year', 'committer_date', 'revision', 'architecture', 'extractor', 'dimacs-transformer'])['dimacs-analyzer-time'].idxmin()]
fig, df_stats = plot(df3, x='committer_date', color='facet', legend_position='right', remove_architecture=False)
show(fig, width=700, height=default_height, margin=dict(l=0, r=0, t=20, b=0))
fig, _ = plot(df3, x='committer_date', color='facet')
fig.update_yaxes(range=[-2.1, 2.6])
annotate_solvers(fig, df3[(df3['architecture'] == 'x86')&(df3['extractor'] == 'KConfigReader')&(df3['dimacs-transformer'] == 'KConfigReader')])
show(fig, 'lessthan', width=250, height=0.7*default_height, margin=dict(l=0, r=0, t=20, b=0))
stats_table(df_stats, None)

In [ ]:
df_current = scope_to(df2, 'sat-museum+competition')
df3 = df_current.loc[df_current.groupby(['year', 'committer_date', 'revision', 'architecture', 'extractor', 'dimacs-transformer'])['dimacs-analyzer-time'].idxmin()]
fig, df_stats = plot(df3, x='committer_date', color='facet', legend_position='right', remove_architecture=False)
show(fig, width=700, height=default_height, margin=dict(l=0, r=0, t=20, b=0))
fig, _ = plot(df3, x='committer_date', color='facet')
fig.update_yaxes(range=[-2.1, 2.6])
annotate_solvers(fig, df3[(df3['architecture'] == 'x86')&(df3['extractor'] == 'KConfigReader')&(df3['dimacs-transformer'] == 'KConfigReader')])
show(fig, 'all', width=250, height=0.7*default_height, margin=dict(l=0, r=0, t=20, b=0))
stats_table(df_stats, None)

In [ ]:
df_current = scope_to(df2, 'FeatureIDE')
fig, df_stats = plot(df_current, x='committer_date', facet_col='extractor', facet_row='dimacs-transformer', color='dimacs-analyzer', color_discrete_sequence=generate_gradient(), legend_position='right', remove_architecture=False)
stats_table(df_stats)
df_current = pd.concat([
    df_current[df_current['year-analyzer'].between(2009, 2010)&df_current['year'].between(2009, 2010)],
    df_current[df_current['year-analyzer'].between(2011, 2013)&df_current['year'].between(2011, 2013)],
    df_current[(df_current['year-analyzer']>=2014)&(df_current['year']>=2014)]  
])
show(fig, width=1500, height=2*default_height, margin=dict(l=0, r=0, t=20, b=0))
fig, df_stats = plot(df_current, x='committer_date', color='facet', legend_position='right', remove_architecture=False)
show(fig, width=700, height=default_height, margin=dict(l=0, r=0, t=20, b=0))
fig, _ = plot(df_current, x='committer_date', color='facet')
fig.update_yaxes(range=[-2.1, 2.6])
show(fig, 'featureide', width=250, height=0.7*default_height, margin=dict(l=0, r=0, t=20, b=0))
stats_table(df_stats, None)

In [ ]:
# df_current = scope_to(df2, 'sat-competition')
# df_total_features = df_features.groupby(['extractor', 'revision']).agg({'#total_features': 'min'}).reset_index()
# df_total_features = pd.merge(df_kconfig[['committer_date', 'revision']].drop_duplicates(), df_total_features)
# print(df_total_features)
# df_total_features = pd.merge(df_total_features, df_current)
# df_total_features = df_total_features[df_total_features['extractor']=='KClause']
# df = df_total_features.sort_values(by='committer_date')
# df = df[['dimacs-analyzer-time', 'dimacs-analyzer', '#total_features']].drop_duplicates()
# # fig = px.scatter(x=df['#total_features'], y=df['dimacs-analyzer-time'], color=df['dimacs-analyzer'], log_y=True)

# # style_scatter(fig, marker_size=3, legend_position=None)
# # show(fig, width=3*330, height=310)
# df_current

df_current = scope_to(df2, 'sat-museum+competition')
print("median SAT solving time: " + str(df_current['dimacs-analyzer-time'].median()/1000000000))
print("percentage of SAT queries over one second: " + str(100-scipy.stats.percentileofscore(df_current['dimacs-analyzer-time']/1000000000, 1)))
print("percentage of SAT queries over half a second: " + str(100-scipy.stats.percentileofscore(df_current['dimacs-analyzer-time']/1000000000, 0.5)))

print()
print("median SAT solving per year:")
for year in range(2002, 2025):
    print(f"{year}: " + str(df_current[df_current['year']==year]['dimacs-analyzer-time'].median()/1000000000))


In [ ]:
# evaluation of experiment_extra.sh
print(f"Statistics for individual solvers")
for query in ['void', '(?:core|dead)', 'core', 'dead', 'partial', 'partial.*\+.*\+', 'partial.*\+.*-', 'partial.*-.*\+', 'partial.*-.*-']:
    df_current = scope_to(df2, 'extra')
    df_current = df_current[df_current['query'].str.contains(query)]
    df_current['architecture'] = df_current['dimacs-analyzer'].str.contains('sat-museum').map({True: 'Winning', False: 'Non-Winning'})
    df_current['extractor'] = df_current['dimacs-analyzer'].str.contains('sat-museum').map({True: 'Winning', False: 'Non-Winning'})
    fig, df_stats = plot(df_current, x='committer_date', facet_col='extractor', color='dimacs-analyzer', color_discrete_sequence=generate_gradient(), legend_position='right', remove_architecture=False, extractor_label='Solver Set', extractor_labels=['Winning', 'Non-Winning'], architecture_label='Solver Set', agg='sum')
    show(fig, f'extra-{query}', width=1500, height=2*default_height, margin=dict(l=0, r=0, t=20, b=0))
    print(f"Statistics for query '{query}':")
    stats_table(df_stats)
    print()

print(f"Statistics for optimal strategy")
for query in ['void', '(?:core|dead)', 'partial']:
    df_current = scope_to(df2, 'extra')
    df_current = df_current[df_current['query'].str.contains(query)]
    df_current['architecture'] = df_current['dimacs-analyzer'].str.contains('sat-museum').map({True: 'Winning', False: 'Non-Winning'})
    df_current['extractor'] = df_current['dimacs-analyzer'].str.contains('sat-museum').map({True: 'Winning', False: 'Non-Winning'})
    df_sum = (
        df_current
        .groupby(['year', 'year-analyzer', 'dimacs-analyzer', 'source-analyzer', 'committer_date', 'revision', 'architecture', 'extractor', 'dimacs-transformer'], as_index=False)
        .agg(total_time=('dimacs-analyzer-time', 'sum'))
    )
    df_sum = df_sum.rename(columns={'total_time': 'dimacs-analyzer-time'})
    df_sum = df_sum[df_sum['year-analyzer'] <= df_sum['year']]
    df3 = df_sum.loc[df_sum.groupby(['year', 'committer_date', 'revision', 'architecture', 'extractor', 'dimacs-transformer'])['dimacs-analyzer-time'].idxmin()]
    fig, df_stats = plot(df3, x='committer_date', remove_architecture=False, architecture_labels=['Non-Winning', 'Winning'], agg='sum')
    show(fig, width=400, height=0.9*default_height, margin=dict(l=0, r=0, t=20, b=0))
    print(f"Statistics for query '{query}':")
    stats_table(df_stats, ['symbol'])
    print()

print(f"Statistics for historic strategy")
for query in ['void', '(?:core|dead)', 'partial']:
    df_current = scope_to(df2, 'extra')
    df_current = df_current[df_current['query'].str.contains(query)]
    df_current['architecture'] = df_current['dimacs-analyzer'].str.contains('sat-museum').map({True: 'Winning', False: 'Non-Winning'})
    df_current['extractor'] = df_current['dimacs-analyzer'].str.contains('sat-museum').map({True: 'Winning', False: 'Non-Winning'})
    df_sum = (
        df_current
        .groupby(['year', 'year-analyzer', 'dimacs-analyzer', 'source-analyzer', 'committer_date', 'revision', 'architecture', 'extractor', 'dimacs-transformer'], as_index=False)
        .agg(total_time=('dimacs-analyzer-time', 'sum'))
    )
    df_sum = df_sum.rename(columns={'total_time': 'dimacs-analyzer-time'})
    df3 = df_sum.loc[df_sum.groupby(['year', 'committer_date', 'revision', 'architecture', 'extractor', 'dimacs-transformer'])['dimacs-analyzer-time'].idxmin()]
    fig, df_stats = plot(df3, x='committer_date', remove_architecture=False, architecture_labels=['Non-Winning', 'Winning'])
    show(fig, width=400, height=0.9*default_height, margin=dict(l=0, r=0, t=20, b=0))
    print(f"Statistics for query '{query}':")
    stats_table(df_stats, ['symbol'])
    print()

In [ ]:
# distribution of satisfiability results in extra experiment
df_current = scope_to(df2, 'extra+timeout')
print(len(df_current[df_current['model-satisfiable']==True])/len(df_current))
print(len(df_current[df_current['model-satisfiable']==False])/len(df_current))
print(len(df_current[df_current['model-satisfiable'].isna()])/len(df_current))
# these results indicate that most queries were satisfiable, and only a small amount of features are core/dead
# a small portion of queries times out

In [ ]:
# determine correctness of solvers by means of majority voting (ignoring NaN values)
# if 22 of 23 solvers agree on (un)satisfiability, we consider the remaining solver to be faulty
df_current = scope_to(df2, 'extra')
df_inconsistent = (
    df_current
    .groupby(['revision', 'architecture', 'extractor', 'dimacs-transformer', 'query'])
    .agg(
        count=('dimacs-analyzer-time', 'count'),
        distinct_count=('model-satisfiable', pd.Series.nunique),
        distinct_values=('model-satisfiable', lambda x: list(x.unique()))
    )
    .reset_index()
    .query('distinct_count != 1')
)
df_inconsistent

group_cols = ['revision', 'architecture', 'extractor', 'dimacs-transformer', 'query']
result_col = 'model-satisfiable'  # your True/False column
solver_col = 'dimacs-analyzer'     # your solver column

def minority_summary_ignore_nan(g):


    g = g if isinstance(g, pd.DataFrame) else pd.DataFrame([g])
    
    # Only consider non-NaN values for majority/minority
    non_nan = g[result_col].dropna()
    
    if non_nan.nunique() <= 1:
        return None  # consistent group or only NaNs

    counts = non_nan.value_counts()
    majority_value = counts.idxmax()
    majority_count = counts.max()
    
    # Minority values (ignore NaN)
    minority_values = [v for v in counts.index if v != majority_value]
    if not minority_values:
        return None
    
    minority_rows = g[g[result_col].isin(minority_values)]
    minority_count = len(minority_rows)
    minority_solvers = ', '.join(minority_rows[solver_col].astype(str))
    
    # Include all group identifiers in the returned DataFrame
    data = {col: g[col].iloc[0] for col in group_cols}  # preserve revision, query, etc.
    data.update({
        'majority_value': majority_value,
        'majority_count': majority_count,
        'minority_value': ', '.join(map(str, minority_values)),
        'minority_count': minority_count,
        'minority_solvers': minority_solvers
    })
    
    return pd.DataFrame([data])

# Apply per group
report = (
    df_current
    .groupby(group_cols, group_keys=False)
    .apply(minority_summary_ignore_nan)
    .reset_index(drop=True)
)
# a manual inspection shows that the only faulty solver is candy-2017

# Explode the minority solvers into a flat Series
bad_solvers = report['minority_solvers'].str.split(', ').explode()

# Keep only rows where the solver is in the bad_solvers
df_bad_solvers = df_current[df_current['dimacs-analyzer'].isin(bad_solvers)]

# Now filter these rows to only include the groups in report
df_bad_calls = df_bad_solvers.merge(
    report[['revision', 'architecture', 'extractor', 'dimacs-transformer', 'query']],
    on=['revision', 'architecture', 'extractor', 'dimacs-transformer', 'query'],
    how='inner'
)
df_bad_calls
df_good_calls = pd.merge(
    df_bad_solvers,
    df_bad_calls,
    how='outer',
    indicator=True
).query('_merge == "left_only"').drop(columns=['_merge'])

len(df_bad_calls)/len(df_bad_solvers)
# this indicates that all solvers agree on all queries, except for candy-2017, which is faulty in 13.3% of its calls, indicating some kind of solver bug
# because it returns correct results in most cases, and does not yield unusual results compared to the other solvers, we keep it in our dataset

In [ ]:
# total time spent in extra experiment
print((scope_to(df2, 'extra+timeout')['dimacs-analyzer-time'] / 1000000000).sum()/3600/24)
# statistics on typical time spent per SAT call (comparison of winning vs. non-winning solvers)
winning = scope_to(df2, 'extra')
winning = winning[winning['dimacs-analyzer'].str.contains('sat-museum')]
nonwinning = scope_to(df2, 'extra')
nonwinning = nonwinning[~nonwinning['dimacs-analyzer'].str.contains('sat-museum')]
(nonwinning['dimacs-analyzer-time'] / 1000000000).describe()/(winning['dimacs-analyzer-time'] / 1000000000).describe()

In [ ]:
# investigate timeouts in extra experiment
df_current=scope_to(df2, 'extra+timeout')
df_current=df_current[df_current['model-satisfiable'].isna()]
df_current[['revision', 'architecture', 'extractor', 'dimacs-transformer', 'query']]
df_current2=scope_to(df2, 'extra')
df_current2=pd.merge(
    df_current,
    df_current2,
    on=['revision', 'architecture', 'extractor', 'dimacs-transformer', 'query'],
    how='inner'
)
# the results show that all timeouts from the local search solvers unitwalk and adaptg2wsat2011 are due to unsatisfiable instances, which is to be expected from this kind of solver
# yalsat, on the other hand, also times out on satisfiable instances, so these are "true timeouts" that we do consider (this is consistent because yalsat is pretty slow overall)
print(df_current2[df_current2['dimacs-analyzer_y']=='sat-museum/kissat-2020'][['dimacs-analyzer_x', 'model-satisfiable_y']].groupby(['dimacs-analyzer_x', 'model-satisfiable_y']).size().reset_index(name='count'))
# the following confirms that indeed yalsat can successfully detect satisfiable instances
df_current=scope_to(df2, 'extra')
df_current[df_current['dimacs-analyzer']==' yalsat-2017'].groupby(['model-satisfiable']).size().reset_index(name='count')


In [ ]:
# compiler comparison (omitted for brevity from section 5.3.6)
for transformer in ['KConfigReader', 'Z3']:
    df_current = pd.concat([scope_to(df2, 'sat-museum'), scope_to(df2, 'sat-competition!=2023!=2024')])
    fig, df_stats = plot(df_current[df_current['dimacs-transformer'] == transformer], x='committer_date', facet_col='source-analyzer', facet_row='extractor', color='dimacs-analyzer', color_discrete_sequence=generate_gradient(), legend_position='right', remove_architecture=False)
    show(fig, width=1500, height=2*default_height, margin=dict(l=0, r=0, t=20, b=0))

In [ ]:
# are solvers getting more complex over time?
df_current = df2[~df2['solver_source_lines_of_code'].isna()]
fig = px.scatter(
    df_current,
    x=df_current['solver_source_lines_of_code_extended'],
    y=df_current['year-analyzer'],
    color='dimacs-analyzer',
    labels={
        'committer_date': 'Year of Feature Model',
        'y': 'SAT Runtime (log<sub>10</sub> s)'
    },
    hover_data=['revision', 'dimacs-analyzer', 'architecture', 'dimacs-transformer', 'extractor']
)
show(fig, width=1000, height=2*default_height)

# test the hypothesis that smaller solvers (in terms of lines of code) have lower SAT runtime
# this does not seem the case (tinisat + black_hole_SAT seem to be outliers)
fig = px.box(
    df_current,
    x=df_current['solver_source_lines_of_code_extended'],
    y=df_current['dimacs-analyzer-time'] / 1000000000,
    log_y=True,
    log_x=True,
    color='dimacs-analyzer',
    labels={
        'committer_date': 'Year of Feature Model',
        'y': 'SAT Runtime (log<sub>10</sub> s)'
    },
    hover_data=['revision', 'dimacs-analyzer', 'architecture', 'dimacs-transformer', 'extractor']
)
show(fig, width=1000, height=2*default_height)